In [48]:
'''
ДАННАЯ ВКЛАДКА - КОПИЯ script_db.py с вставленным sql-запросом

'''

# Данный python-скрипт имитирует запрос к БД
# Напишите ваш SQL-запрос в query и запустите данный python-скрипт для получения результата
# Перед запуском скрипта установите библиотеку duckdb

# Установка библиотеки duckdb
# pip install duckdb duckdb-engine

# Импорт библиотек
import pandas as pd
import duckdb
import numpy as np

# Задание таблиц БД
users = pd.read_csv('users.csv')
course_users = pd.read_csv('course_users.csv')
courses = pd.read_csv('courses.csv')
course_types = pd.read_csv('course_types.csv')
lessons = pd.read_csv('lessons.csv')
subjects = pd.read_csv('subjects.csv')
cities = pd.read_csv('cities.csv')
homework_done = pd.read_csv('homework_done.csv')
homework = pd.read_csv('homework.csv')
homework_lessons = pd.read_csv('homework_lessons.csv')
user_roles = pd.read_csv('user_roles.csv') 

# Задание SQL-запроса
query = """
SELECT  
c.id as course_id
, c.name course_name
, s.name subject_name
, s.project subject_type, 
ct.name course_type
, c.starts_at::date course_start_date, --перевел starts_at в тип date
u.id user_id,
u.last_name user_last_name,
ci.name as city_name,
case when cu.active=1 then 'Активный' else 'Неактивный' end as active , -- так почеловечнее
cu.created_at::date open_date, -- Возможно, это дата открытия курса ученику, не совсем ясно, что под этим подразумевалось
coalesce(FLOOR(available_lessons::decimal/c.lessons_in_month), 0) as avalilable_months, -- число доступных месяцев взял как отношение открытых для ученика уроков к числу уроков в месяц и округлил в меньшую сторону
count(homework_id) over (partition by u.id) as user_hws_done --сделал оконку, чтобы не надо было группировать и допускать ошибки, считаю кол-во сданных дз на одного ученика

FROM courses c 
 join subjects s on c.subject_id=s.id
 join course_types ct on c.course_type_id=ct.id
 join course_users cu on c.id=cu.course_id
 join users u on cu.user_id=u.id
 join cities ci on u.city_id=ci.id
 join homework_done hd on u.id=hd.user_id

order by c.id

"""


# Выполнение SQL-запроса
df_result = duckdb.query(query).to_df()

# Вывод результата
display(df_result)


,course_id,course_name,subject_name,subject_type,course_type,course_start_date,user_id,user_last_name,city_name,active,open_date,avalilable_months,user_hws_done
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10011,Ищенко,Анива,Неактивный,2024-07-26,0.0,143
1,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10011,Ищенко,Анива,Неактивный,2024-07-26,0.0,143
2,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10011,Ищенко,Анива,Неактивный,2024-07-26,0.0,143
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10011,Ищенко,Анива,Неактивный,2024-07-26,0.0,143
4,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10011,Ищенко,Анива,Неактивный,2024-07-26,0.0,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
291953,104,СПЕЦКУРС ПО КАРТАМ,История,ЕГЭ,Спецкурс,2024-11-10,33947,Шерстнева,Верхняя Салда,Активный,2024-11-10,0.0,40
291954,104,СПЕЦКУРС ПО КАРТАМ,История,ЕГЭ,Спецкурс,2024-11-10,33947,Шерстнева,Верхняя Салда,Активный,2024-11-10,0.0,40
291955,104,СПЕЦКУРС ПО КАРТАМ,История,ЕГЭ,Спецкурс,2024-11-10,33947,Шерстнева,Верхняя Салда,Активный,2024-11-10,0.0,40
291956,104,СПЕЦКУРС ПО КАРТАМ,История,ЕГЭ,Спецкурс,2024-11-10,33947,Шерстнева,Верхняя Салда,Активный,2024-11-10,0.0,40


In [49]:
# делаю csv-файл для проверки
# df_result.to_csv('itog.csv', encoding = 'cp1251', columns=['course_id','course_name','subject_name','subject_type','course_type','course_start_date','user_id','user_last_name','city_name','active','open_date','avalilable_months','user_hws_done'])

In [50]:
#ЗАДАНИЕ 2
""" 
С использованием python-скриптов обработать датасет, подготовленный в Задаче 1:
1. Проверить датасет на: дубликаты, пропуски, типы данных, аномальные значения. Произвести
предобработку датасета, если это необходимо
2. Поделить всех учеников на волны
Критерии волн:
• 0 волна – присоединились на курс до даты старта курса включительно
• 1 волна - присоединились на курс в течение 1 недели после даты старта курса включительно
• 2 волна - присоединились на курс в течение от 1 до 2 недель включительно после даты старта курса
• 3 волна - присоединились на курс в течение от 2 до 3 недель включительно после даты старта курса
• 4 волна - присоединились на курс в течение от 3 до 4 недель включительно после даты старта курса
• 5 волна - присоединились на курс от 4 недель после даты старта курса
Результат, который нужно прислать по задаче:
• Jupyter Notebook в формате файла .ipynb. В тетрадке оставляйте комментарии с пояснениями своих
действий

"""




duplicates = df_result.duplicated().sum()
if duplicates > 0:
    df_result = df_result.drop_duplicates() # Проверяем таблицу на дубликаты, считаем их и удаляем их


df_result.fillna('-', inplace=True) # заполняем NaN значения прочерком

# Я это уже делаю в sql-запросе, поэтому нет необходимости переводить эти столбцы в тип даты
'''
df['course_start_date']= pd.to_datetime(df['course_start_date']) 
df['open_date']= pd.to_datetime(df['open_date'])
'''


# создадим когорты по волнам
def assign_wave(row):
    if row['open_date'] <= row['course_start_date']: # если столбец регистрации произошел раньше, чем начало курса, то 0 волна
        return '0 волна'
    elif row['course_start_date'] < row['open_date'] <= row['course_start_date'] + pd.Timedelta(weeks=1): #если столбец рег. между началом курса и 1 неделей, то 1 волна
        return '1 волна'
    elif row['course_start_date'] + pd.Timedelta(weeks=1) < row['open_date'] <= row['course_start_date'] + pd.Timedelta(weeks=2):# и так далее
        return '2 волна'
    elif row['course_start_date'] + pd.Timedelta(weeks=2) < row['open_date'] <= row['course_start_date'] + pd.Timedelta(weeks=3):
        return '3 волна'
    elif row['course_start_date'] + pd.Timedelta(weeks=3) < row['open_date'] <= row['course_start_date'] + pd.Timedelta(weeks=4):
        return '4 волна'
    else:
        return '5 волна'

# Применение функции для определения волн
df_result['wave'] = df_result.apply(assign_wave, axis=1)


# Сохранение обработанного датасета
df_result.to_csv('processed_dataset.csv',encoding = 'cp1251', columns=['course_id','course_name','subject_name','subject_type','course_type','course_start_date','user_id','user_last_name','city_name','active','open_date','avalilable_months','user_hws_done', 'wave'])  # Сохраните обработанный датасет в новый файл


In [51]:
df_result

,course_id,course_name,subject_name,subject_type,course_type,course_start_date,user_id,user_last_name,city_name,active,open_date,avalilable_months,user_hws_done,wave
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10011,Ищенко,Анива,Неактивный,2024-07-26,0.0,143,0 волна
11,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10066,Баринова,Ижевск,Активный,2024-09-15,3.0,533,0 волна
52,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10175,Евсенко,Элиста,Активный,2024-07-29,2.0,186,0 волна
83,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10263,Кондратьева,Нижнекамск,Активный,2024-07-30,2.0,80,0 волна
99,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10280,Зуева,Санкт-Петербург,Активный,2024-07-30,1.0,64,0 волна
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291884,104,СПЕЦКУРС ПО КАРТАМ,История,ЕГЭ,Спецкурс,2024-11-10,33111,Кара,Санкт-Петербург,Активный,2024-11-10,0.0,10,0 волна
291886,104,СПЕЦКУРС ПО КАРТАМ,История,ЕГЭ,Спецкурс,2024-11-10,33153,Ка,Улан-Удэ,Активный,2024-11-10,0.0,70,0 волна
291900,104,СПЕЦКУРС ПО КАРТАМ,История,ЕГЭ,Спецкурс,2024-11-10,33698,Истомин,Волоколамск,Активный,2024-11-10,0.0,234,0 волна
291939,104,СПЕЦКУРС ПО КАРТАМ,История,ЕГЭ,Спецкурс,2024-11-10,33915,Софина,Нягань,Активный,2024-11-10,0.0,54,0 волна
